In [ ]:
!pip install pydub
!pip install sklearn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import pandas as pd
from pydub import AudioSegment
from sklearn.preprocessing import LabelEncoder
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
zip_file_path = '/content/drive/My Drive/en.zip'

import zipfile
import os

# Specify the extraction directory
extract_dir = '/content/drive/My Drive/data/extracted_files-2/'

# Create the directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
extracted_files = os.listdir(extract_dir)
print(extracted_files)


['en']


In [ ]:


# Load the metadata file
metadata_path = os.path.join(extract_dir, 'en/validated.tsv')
# metadata_path = 'validated.tsv'  # Replace with the correct path to your 'validated.tsv' file
metadata = pd.read_csv(metadata_path, sep='\t')  # Read the TSV file into a DataFrame

# Display the first few rows of the DataFrame to understand its structure
print(metadata.head())


                                           client_id  \
0  01e8ea298cdecf26e273f5baac3915eb992c493f229686...   
1  02cbc1fe01fc67fa72c6e067fbe020399082efbeb57a2b...   
2  03b62f72067ec967c423852bef03d1b61e63c156d86f6e...   
3  05112cb5965431bbd47abd29b4faea9fb009b5a2e320e0...   
4  05d33ad00cc2754da8e542a33a5255f9346535ef1d8619...   

                           path  \
0  common_voice_en_39751075.mp3   
1  common_voice_en_39589864.mp3   
2  common_voice_en_40087973.mp3   
3  common_voice_en_39587246.mp3   
4  common_voice_en_40117514.mp3   

                                         sentence_id  \
0  e5e7d4694b7160add018a08876327f254690c1ab4c39ea...   
1  e3e7c913ce32a3b5a58dda5fa1d855f2529ed36e1fa33f...   
2  e90c361c9684d01d31bc6e8df3060bc97e536ca707bef4...   
3  e3a1d0662e080f880a899b5a5226af16acf61360b1128e...   
4  e9475052b6e625f8c5890389e4ffc17a1078dec1483592...   

                                            sentence sentence_domain  \
0  Madin was a significant figure of post-w

In [ ]:
# Check if the 'accent' column is present in the dataset
if 'accents' in metadata.columns:
    # Extract unique accents
    unique_accents = metadata['accents'].dropna().unique()  # Drop NaN values and find unique accents
    print("Unique accents in the dataset:")
    print(unique_accents)  # Display the unique accents
else:
    print("The 'accent' column is not found in the metadata.")


Unique accents in the dataset:
['United States English,New York English' 'Scottish English'
 'Australian English' 'United States English' 'Northern English'
 'United States English,Transgender,California'
 'India and South Asia (India, Pakistan, Sri Lanka)'
 'Scottish English,Posh Scottish English'
 'United States English,England English,Transatlantic English'
 'German English,German native speaker' 'Canadian English'
 'Northern Irish,belfast ' 'Icelandic' 'Canadian English,Lisp'
 'United States English,midwestern united states' 'Zambia'
 'England English'
 'Midwestern United States - Chicago,Tinged with southern accent,United States English'
 'Slavic English' 'Spanish,american'
 'United States English,England English'
 'United States English,Hungarian English (English spoken by a Hungarian national)'
 'United States English,Philadelphia'
 'England English,Surrey accent, Southern English.'
 'United States English,Danish'
 'England English,Western European non-native English'
 'Irish En

In [ ]:
# Step 2: Remove rows with NaN or non-string values from the 'accents' column
metadata = metadata[metadata['accents'].apply(lambda x: isinstance(x, str))]  # Keep only rows where 'accents' is a string

# Step 3: Define the function to replace accents starting with 'United States' with 'American English'
def replace_united_states_with_american(accent):
    accents = accent.split(',')  # Split by comma to handle multiple accents
    updated_accents = []

    for acc in accents:
        acc = acc.strip()  # Remove leading and trailing whitespace
        if acc.startswith('United States'):
            updated_accents.append('American English')  # Replace with 'American English'
        else:
            updated_accents.append(acc)  # Keep the original accent

    return ', '.join(updated_accents)  # Join accents back with commas

# Step 4: Apply the function to the 'accents' column in your DataFrame
metadata['accents'] = metadata['accents'].apply(replace_united_states_with_american)

# Step 5: Display the updated DataFrame to verify the changes
print("Cleaned and Updated Accents in Metadata:")
print(metadata['accents'].unique())  # Display unique values to check the replacement

metadata['accents'].head(5)

Cleaned and Updated Accents in Metadata:
['American English, New York English' 'Scottish English'
 'Australian English' 'American English' 'Northern English'
 'American English, Transgender, California'
 'India and South Asia (India, Pakistan, Sri Lanka)'
 'Scottish English, Posh Scottish English'
 'American English, England English, Transatlantic English'
 'German English, German native speaker' 'Canadian English'
 'Northern Irish, belfast' 'Icelandic' 'Canadian English, Lisp'
 'American English, midwestern united states' 'Zambia' 'England English'
 'Midwestern United States - Chicago, Tinged with southern accent, American English'
 'Slavic English' 'Spanish, american' 'American English, England English'
 'American English, Hungarian English (English spoken by a Hungarian national)'
 'American English, Philadelphia'
 'England English, Surrey accent, Southern English.'
 'American English, Danish'
 'England English, Western European non-native English'
 'Irish English, England English'


<ipython-input-8-542c03067f7d>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata['accents'] = metadata['accents'].apply(replace_united_states_with_american)


,accents
0,"American English, New York English"
2,Scottish English
4,Australian English
5,American English
6,Northern English


In [ ]:

# Step 1: Filter the 'accents' column to select rows starting with 'American English'
american_accents = metadata[metadata['accents'].str.startswith('American English', na=False)]  # Filter rows

# Step 2: Print the filtered accents
print("Accents Starting with 'American English':")
print(american_accents['accents'].unique())  # Display unique accents that start with 'American English'


Accents Starting with 'American English':
['American English, New York English' 'American English'
 'American English, Transgender, California'
 'American English, England English, Transatlantic English'
 'American English, midwestern united states'
 'American English, England English'
 'American English, Hungarian English (English spoken by a Hungarian national)'
 'American English, Philadelphia' 'American English, Danish'
 'American English, California' 'American English, Midwestern'
 'American English, portuguese' 'American English, Minnesotan'
 'American English, Midwestern, Low, Demure'
 'American English, Canadian English'
 'American English, A midwestern accent, specific to the St. Louis area.'
 'American English, vietnamese accent' 'American English, Mid Atlantic'
 'American English, Sacramento' 'American English, Midwestern, Michigan'
 'American English, Eastern United States'
 'American English, midwestern Missourian drawl'
 'American English, Foreign accent' 'American Englis

In [ ]:
# Example: To check the count of 'American English'
accent_to_check = 'American English'  # Replace with the accent you want to check

# Count the occurrences of 'American English' in the 'accents' column
accent_count = metadata['accents'].value_counts().get(accent_to_check, 0)

# Print the count of the specified accent
print(f"Count of '{accent_to_check}': {accent_count}")


Count of 'American English': 479


In [ ]:


# Step 1: Define the function to generalize accents starting with 'American English'
def generalize_american_english(accent):
    if accent.startswith('American English'):
        return 'American English'  # Generalize to 'American English'
    return accent  # Keep other accents unchanged

# Step 2: Apply the function to the 'accents' column in your DataFrame
metadata['accents'] = metadata['accents'].apply(generalize_american_english)

# Step 3: Print the unique accents to verify the changes
print("Generalized Accents in Metadata:")
print(metadata['accents'].unique())  # Display unique values after generalization


Generalized Accents in Metadata:
['American English' 'Scottish English' 'Australian English'
 'Northern English' 'India and South Asia (India, Pakistan, Sri Lanka)'
 'Scottish English, Posh Scottish English'
 'German English, German native speaker' 'Canadian English'
 'Northern Irish, belfast' 'Icelandic' 'Canadian English, Lisp' 'Zambia'
 'England English'
 'Midwestern United States - Chicago, Tinged with southern accent, American English'
 'Slavic English' 'Spanish, american'
 'England English, Surrey accent, Southern English.'
 'England English, Western European non-native English'
 'Irish English, England English'
 'Latin American accent influenced by American English'
 'England English, London English' 'New Zealand English'
 'Algeria, Algerian'
 'England English, Souther English, long-term resident in Scotland'
 'German English, Sick' 'Italian'
 'England English, International Southern/London' 'Dutch English'
 'Southern African (South Africa, Zimbabwe, Namibia)' 'Irish English'
 '

In [ ]:
# Step 1: Define the function to generalize accents starting with 'American English'
def generalize_american_english(accent):
    if accent.startswith('England English'):
        return 'British English'  # Generalize to 'American English'
    return accent  # Keep other accents unchanged

# Step 2: Apply the function to the 'accents' column in your DataFrame
metadata['accents'] = metadata['accents'].apply(generalize_american_english)

# Step 3: Print the unique accents to verify the changes
print("Generalized Accents in Metadata:")
print(metadata['accents'].unique())  # Display unique values after generalization


Generalized Accents in Metadata:
['American English' 'Scottish English' 'Australian English'
 'Northern English' 'India and South Asia (India, Pakistan, Sri Lanka)'
 'Scottish English, Posh Scottish English'
 'German English, German native speaker' 'Canadian English'
 'Northern Irish, belfast' 'Icelandic' 'Canadian English, Lisp' 'Zambia'
 'British English'
 'Midwestern United States - Chicago, Tinged with southern accent, American English'
 'Slavic English' 'Spanish, american' 'Irish English, England English'
 'Latin American accent influenced by American English'
 'New Zealand English' 'Algeria, Algerian' 'German English, Sick'
 'Italian' 'Dutch English'
 'Southern African (South Africa, Zimbabwe, Namibia)' 'Irish English'
 'Slovenian, American English' 'Chinese-Australian English'
 'British, England English' 'Hungarian' 'Hong Kong English' 'Russian'
 'German' 'German English'
 'Bengali English, India and South Asia (India, Pakistan, Sri Lanka)'
 'american' 'Scottish English, highlan

In [ ]:
# Step 1: Define the function to generalize accents starting with 'American English'
def generalize_american_english(accent):
    if accent.startswith('British English'):
        return 'British English'  # Generalize to 'American English'
    return accent  # Keep other accents unchanged

# Step 2: Apply the function to the 'accents' column in your DataFrame
metadata['accents'] = metadata['accents'].apply(generalize_american_english)

# Step 3: Print the unique accents to verify the changes
print("Generalized Accents in Metadata:")
print(metadata['accents'].unique())  # Display unique values after generalization


Generalized Accents in Metadata:
['American English' 'Scottish English' 'Australian English'
 'Northern English' 'India and South Asia (India, Pakistan, Sri Lanka)'
 'Scottish English, Posh Scottish English'
 'German English, German native speaker' 'Canadian English'
 'Northern Irish, belfast' 'Icelandic' 'Canadian English, Lisp' 'Zambia'
 'British English'
 'Midwestern United States - Chicago, Tinged with southern accent, American English'
 'Slavic English' 'Spanish, american' 'Irish English, England English'
 'Latin American accent influenced by American English'
 'New Zealand English' 'Algeria, Algerian' 'German English, Sick'
 'Italian' 'Dutch English'
 'Southern African (South Africa, Zimbabwe, Namibia)' 'Irish English'
 'Slovenian, American English' 'Chinese-Australian English'
 'British, England English' 'Hungarian' 'Hong Kong English' 'Russian'
 'German' 'German English'
 'Bengali English, India and South Asia (India, Pakistan, Sri Lanka)'
 'american' 'Scottish English, highlan

In [ ]:

# Step 1: Define the function to generalize accents starting with 'American English'
def generalize_american_english(accent):
    if accent.startswith('Canadian English'):
        return 'Canadian English'  # Generalize to 'American English'
    return accent  # Keep other accents unchanged

# Step 2: Apply the function to the 'accents' column in your DataFrame
metadata['accents'] = metadata['accents'].apply(generalize_american_english)

# Step 3: Print the unique accents to verify the changes
print("Generalized Accents in Metadata:")
print(metadata['accents'].unique())  # Display unique values after generalization


Generalized Accents in Metadata:
['American English' 'Scottish English' 'Australian English'
 'Northern English' 'India and South Asia (India, Pakistan, Sri Lanka)'
 'Scottish English, Posh Scottish English'
 'German English, German native speaker' 'Canadian English'
 'Northern Irish, belfast' 'Icelandic' 'Zambia' 'British English'
 'Midwestern United States - Chicago, Tinged with southern accent, American English'
 'Slavic English' 'Spanish, american' 'Irish English, England English'
 'Latin American accent influenced by American English'
 'New Zealand English' 'Algeria, Algerian' 'German English, Sick'
 'Italian' 'Dutch English'
 'Southern African (South Africa, Zimbabwe, Namibia)' 'Irish English'
 'Slovenian, American English' 'Chinese-Australian English'
 'British, England English' 'Hungarian' 'Hong Kong English' 'Russian'
 'German' 'German English'
 'Bengali English, India and South Asia (India, Pakistan, Sri Lanka)'
 'american' 'Scottish English, highland, east highland' 'Turkish

In [ ]:
# Step 1: Define the function to generalize accents starting with 'American English'
def generalize_american_english(accent):
    if accent.startswith('Northern English'):
        return 'Northern English'  # Generalize to 'American English'
    return accent  # Keep other accents unchanged

# Step 2: Apply the function to the 'accents' column in your DataFrame
metadata['accents'] = metadata['accents'].apply(generalize_american_english)

# Step 3: Print the unique accents to verify the changes
print("Generalized Accents in Metadata:")
print(metadata['accents'].unique())  # Display unique values after generalization


Generalized Accents in Metadata:
['American English' 'Scottish English' 'Australian English'
 'Northern English' 'India and South Asia (India, Pakistan, Sri Lanka)'
 'Scottish English, Posh Scottish English'
 'German English, German native speaker' 'Canadian English'
 'Northern Irish, belfast' 'Icelandic' 'Zambia' 'British English'
 'Midwestern United States - Chicago, Tinged with southern accent, American English'
 'Slavic English' 'Spanish, american' 'Irish English, England English'
 'Latin American accent influenced by American English'
 'New Zealand English' 'Algeria, Algerian' 'German English, Sick'
 'Italian' 'Dutch English'
 'Southern African (South Africa, Zimbabwe, Namibia)' 'Irish English'
 'Slovenian, American English' 'Chinese-Australian English'
 'British, England English' 'Hungarian' 'Hong Kong English' 'Russian'
 'German' 'German English'
 'Bengali English, India and South Asia (India, Pakistan, Sri Lanka)'
 'american' 'Scottish English, highland, east highland' 'Turkish

In [ ]:
# Step 1: Define the function to generalize accents starting with 'American English'
def generalize_american_english(accent):
    if accent.startswith('India and South Asia'):
        return 'South Asian English'  # Generalize to 'American English'
    return accent  # Keep other accents unchanged

# Step 2: Apply the function to the 'accents' column in your DataFrame
metadata['accents'] = metadata['accents'].apply(generalize_american_english)

# Step 3: Print the unique accents to verify the changes
print("Generalized Accents in Metadata:")
print(metadata['accents'].unique())  # Display unique values after generalization


Generalized Accents in Metadata:
['American English' 'Scottish English' 'Australian English'
 'Northern English' 'South Asian English'
 'Scottish English, Posh Scottish English'
 'German English, German native speaker' 'Canadian English'
 'Northern Irish, belfast' 'Icelandic' 'Zambia' 'British English'
 'Midwestern United States - Chicago, Tinged with southern accent, American English'
 'Slavic English' 'Spanish, american' 'Irish English, England English'
 'Latin American accent influenced by American English'
 'New Zealand English' 'Algeria, Algerian' 'German English, Sick'
 'Italian' 'Dutch English'
 'Southern African (South Africa, Zimbabwe, Namibia)' 'Irish English'
 'Slovenian, American English' 'Chinese-Australian English'
 'British, England English' 'Hungarian' 'Hong Kong English' 'Russian'
 'German' 'German English'
 'Bengali English, India and South Asia (India, Pakistan, Sri Lanka)'
 'american' 'Scottish English, highland, east highland' 'Turkish'
 'Welsh English, Shropshire'

In [ ]:
# Step 1: Define the function to generalize accents starting with 'American English'
def generalize_american_english(accent):
    if accent.startswith('German'):
        return 'European English'  # Generalize to 'American English'
    return accent  # Keep other accents unchanged

# Step 2: Apply the function to the 'accents' column in your DataFrame
metadata['accents'] = metadata['accents'].apply(generalize_american_english)

# Step 3: Print the unique accents to verify the changes
print("Generalized Accents in Metadata:")
print(metadata['accents'].unique())  # Display unique values after generalization


Generalized Accents in Metadata:
['American English' 'Scottish English' 'Australian English'
 'Northern English' 'South Asian English'
 'Scottish English, Posh Scottish English' 'European English'
 'Canadian English' 'Northern Irish, belfast' 'Icelandic' 'Zambia'
 'British English'
 'Midwestern United States - Chicago, Tinged with southern accent, American English'
 'Slavic English' 'Spanish, american' 'Irish English, England English'
 'Latin American accent influenced by American English'
 'New Zealand English' 'Algeria, Algerian' 'Italian' 'Dutch English'
 'Southern African (South Africa, Zimbabwe, Namibia)' 'Irish English'
 'Slovenian, American English' 'Chinese-Australian English'
 'British, England English' 'Hungarian' 'Hong Kong English' 'Russian'
 'Bengali English, India and South Asia (India, Pakistan, Sri Lanka)'
 'american' 'Scottish English, highland, east highland' 'Turkish'
 'Welsh English, Shropshire' 'Chinese English, British English'
 'Немного русский акцент произношени

In [ ]:

# Step 1: Define the function to generalize accents containing 'German'
def generalize_to_european_english(accent):
    if 'Scottish' in accent:  # Check if 'German' is anywhere in the accent string
        return 'British English'  # Generalize to 'European English'
    return accent  # Keep other accents unchanged

# Step 2: Apply the function to the 'accents' column in your DataFrame
metadata['accents'] = metadata['accents'].apply(generalize_to_european_english)

# Step 3: Print the unique accents to verify the changes
print("Generalized Accents in Metadata:")
print(metadata['accents'].unique())  # Display unique values after generalization


Generalized Accents in Metadata:
['American English' 'British English' 'Australian English'
 'Northern English' 'South Asian English' 'European English'
 'Canadian English' 'Northern Irish, belfast' 'Icelandic' 'Zambia'
 'Midwestern United States - Chicago, Tinged with southern accent, American English'
 'Slavic English' 'Spanish, american' 'Irish English, England English'
 'Latin American accent influenced by American English'
 'New Zealand English' 'Algeria, Algerian' 'Italian' 'Dutch English'
 'Southern African (South Africa, Zimbabwe, Namibia)' 'Irish English'
 'Slovenian, American English' 'Chinese-Australian English'
 'British, England English' 'Hungarian' 'Hong Kong English' 'Russian'
 'Bengali English, India and South Asia (India, Pakistan, Sri Lanka)'
 'american' 'Turkish' 'Welsh English, Shropshire'
 'Chinese English, British English'
 'Немного русский акцент произношения, съедание слов' 'Taiwanese English'
 'Blurpy' 'Slavic, Russian, Cyrilic' 'North Africa'
 'Korean American

In [ ]:

# Step 1: Define the function to generalize accents containing 'German'
def generalize_to_european_english(accent):
    if 'India' in accent:  # Check if 'German' is anywhere in the accent string
        return 'South Asia English'  # Generalize to 'European English'
    return accent  # Keep other accents unchanged

# Step 2: Apply the function to the 'accents' column in your DataFrame
metadata['accents'] = metadata['accents'].apply(generalize_to_european_english)

# Step 3: Print the unique accents to verify the changes
print("Generalized Accents in Metadata:")
print(metadata['accents'].unique())  # Display unique values after generalization


Generalized Accents in Metadata:
['American English' 'British English' 'Australian English'
 'Northern English' 'South Asian English' 'European English'
 'Canadian English' 'Northern Irish, belfast' 'Icelandic' 'Zambia'
 'Midwestern United States - Chicago, Tinged with southern accent, American English'
 'Slavic English' 'Spanish, american' 'Irish English, England English'
 'Latin American accent influenced by American English'
 'New Zealand English' 'Algeria, Algerian' 'Italian' 'Dutch English'
 'Southern African (South Africa, Zimbabwe, Namibia)' 'Irish English'
 'Slovenian, American English' 'Chinese-Australian English'
 'British, England English' 'Hungarian' 'Hong Kong English' 'Russian'
 'South Asia English' 'american' 'Turkish' 'Welsh English, Shropshire'
 'Chinese English, British English'
 'Немного русский акцент произношения, съедание слов' 'Taiwanese English'
 'Blurpy' 'Slavic, Russian, Cyrilic' 'North Africa'
 'Korean American, Korean American English'
 'Eastern European Eng

In [ ]:


# Step 1: Define the function to generalize accents containing 'German'
def generalize_to_european_english(accent):
    if 'United States' in accent:  # Check if 'German' is anywhere in the accent string
        return 'American English'  # Generalize to 'European English'
    return accent  # Keep other accents unchanged

# Step 2: Apply the function to the 'accents' column in your DataFrame
metadata['accents'] = metadata['accents'].apply(generalize_to_european_english)

# Step 3: Print the unique accents to verify the changes
print("Generalized Accents in Metadata:")
print(metadata['accents'].unique())  # Display unique values after generalization


Generalized Accents in Metadata:
['American English' 'British English' 'Australian English'
 'Northern English' 'South Asian English' 'European English'
 'Canadian English' 'Northern Irish, belfast' 'Icelandic' 'Zambia'
 'Slavic English' 'Spanish, american' 'Irish English, England English'
 'Latin American accent influenced by American English'
 'New Zealand English' 'Algeria, Algerian' 'Italian' 'Dutch English'
 'Southern African (South Africa, Zimbabwe, Namibia)' 'Irish English'
 'Slovenian, American English' 'Chinese-Australian English'
 'British, England English' 'Hungarian' 'Hong Kong English' 'Russian'
 'South Asia English' 'american' 'Turkish' 'Welsh English, Shropshire'
 'Chinese English, British English'
 'Немного русский акцент произношения, съедание слов' 'Taiwanese English'
 'Blurpy' 'Slavic, Russian, Cyrilic' 'North Africa'
 'Korean American, Korean American English'
 'Eastern European English, bulgarian english'
 'Non Native - French speaker' 'Swedish'
 'Transfemale - US 

In [ ]:


# Step 1: Define the function to generalize accents containing 'German'
def generalize_to_european_english(accent):
    if 'Chinese' in accent:  # Check if 'German' is anywhere in the accent string
        return 'East Asian English'  # Generalize to 'European English'
    return accent  # Keep other accents unchanged

# Step 2: Apply the function to the 'accents' column in your DataFrame
metadata['accents'] = metadata['accents'].apply(generalize_to_european_english)

# Step 3: Print the unique accents to verify the changes
print("Generalized Accents in Metadata:")
print(metadata['accents'].unique())  # Display unique values after generalization


Generalized Accents in Metadata:
['American English' 'British English' 'Australian English'
 'Northern English' 'South Asian English' 'European English'
 'Canadian English' 'Northern Irish, belfast' 'Icelandic' 'Zambia'
 'Slavic English' 'Spanish, american' 'Irish English, England English'
 'Latin American accent influenced by American English'
 'New Zealand English' 'Algeria, Algerian' 'Italian' 'Dutch English'
 'Southern African (South Africa, Zimbabwe, Namibia)' 'Irish English'
 'Slovenian, American English' 'East Asian English'
 'British, England English' 'Hungarian' 'Hong Kong English' 'Russian'
 'South Asia English' 'american' 'Turkish' 'Welsh English, Shropshire'
 'Немного русский акцент произношения, съедание слов' 'Taiwanese English'
 'Blurpy' 'Slavic, Russian, Cyrilic' 'North Africa'
 'Korean American, Korean American English'
 'Eastern European English, bulgarian english'
 'Non Native - French speaker' 'Swedish'
 'Transfemale - US English, American English, Transgender Woman

In [ ]:


# Step 1: Define the function to generalize accents containing 'German'
def generalize_to_european_english(accent):
    if 'African' in accent:  # Check if 'German' is anywhere in the accent string
        return 'African English'  # Generalize to 'European English'
    return accent  # Keep other accents unchanged

# Step 2: Apply the function to the 'accents' column in your DataFrame
metadata['accents'] = metadata['accents'].apply(generalize_to_european_english)

# Step 3: Print the unique accents to verify the changes
print("Generalized Accents in Metadata:")
print(metadata['accents'].unique())  # Display unique values after generalization


Generalized Accents in Metadata:
['American English' 'British English' 'Australian English'
 'Northern English' 'South Asian English' 'European English'
 'Canadian English' 'Northern Irish, belfast' 'Icelandic' 'Zambia'
 'Slavic English' 'Spanish, american' 'Irish English, England English'
 'Latin American accent influenced by American English'
 'New Zealand English' 'Algeria, Algerian' 'Italian' 'Dutch English'
 'African English' 'Irish English' 'Slovenian, American English'
 'East Asian English' 'British, England English' 'Hungarian'
 'Hong Kong English' 'Russian' 'South Asia English' 'american' 'Turkish'
 'Welsh English, Shropshire'
 'Немного русский акцент произношения, съедание слов' 'Taiwanese English'
 'Blurpy' 'Slavic, Russian, Cyrilic' 'North Africa'
 'Korean American, Korean American English'
 'Eastern European English, bulgarian english'
 'Non Native - French speaker' 'Swedish'
 'Transfemale - US English, American English, Transgender Woman - US English'
 'midwestern united 

In [ ]:


# Step 1: Define the function to generalize accents containing 'African' or 'South African'
def generalize_to_african_english(accent):
    # Check if 'African' or 'South African' is anywhere in the accent string
    if 'Hong Kong' in accent or 'Taiwanese' in accent or 'Korean' in accent:
        return 'East Asian English'  # Generalize to 'African English'
    return accent  # Keep other accents unchanged

# Step 2: Apply the function to the 'accents' column in your DataFrame
metadata['accents'] = metadata['accents'].apply(generalize_to_african_english)

# Step 3: Print the unique accents to verify the changes
print("Generalized Accents in Metadata:")
print(metadata['accents'].unique())  # Display unique values after generalization


Generalized Accents in Metadata:
['American English' 'British English' 'Australian English'
 'Northern English' 'South Asian English' 'European English'
 'Canadian English' 'Northern Irish, belfast' 'Icelandic' 'Zambia'
 'Slavic English' 'Spanish, american' 'Irish English, England English'
 'Latin American accent influenced by American English'
 'New Zealand English' 'Algeria, Algerian' 'Italian' 'Dutch English'
 'African English' 'Irish English' 'Slovenian, American English'
 'East Asian English' 'British, England English' 'Hungarian' 'Russian'
 'South Asia English' 'american' 'Turkish' 'Welsh English, Shropshire'
 'Немного русский акцент произношения, съедание слов' 'Blurpy'
 'Slavic, Russian, Cyrilic' 'North Africa'
 'Eastern European English, bulgarian english'
 'Non Native - French speaker' 'Swedish'
 'Transfemale - US English, American English, Transgender Woman - US English'
 'midwestern united states' 'European accent' 'Welsh English'
 'Transatlantic, Danish English' 'Singapore

In [ ]:

# Step 1: Define the function to generalize accents containing 'African' or 'South African'
def generalize_to_african_english(accent):
    # Check if 'African' or 'South African' is anywhere in the accent string
    if 'Dutch' in accent or 'Hungarian' in accent or 'Russian' in accent or 'Bulgarian' in accent or 'Danish' in accent or 'Swedish' in accent or 'Spanish' in accent or 'French' in accent:
        return 'European English'  # Generalize to 'African English'
    return accent  # Keep other accents unchanged

# Step 2: Apply the function to the 'accents' column in your DataFrame
metadata['accents'] = metadata['accents'].apply(generalize_to_african_english)

# Step 3: Print the unique accents to verify the changes
print("Generalized Accents in Metadata:")
print(metadata['accents'].unique())  # Display unique values after generalization


Generalized Accents in Metadata:
['American English' 'British English' 'Australian English'
 'Northern English' 'South Asian English' 'European English'
 'Canadian English' 'Northern Irish, belfast' 'Icelandic' 'Zambia'
 'Slavic English' 'Irish English, England English'
 'Latin American accent influenced by American English'
 'New Zealand English' 'Algeria, Algerian' 'Italian' 'African English'
 'Irish English' 'Slovenian, American English' 'East Asian English'
 'British, England English' 'South Asia English' 'american' 'Turkish'
 'Welsh English, Shropshire'
 'Немного русский акцент произношения, съедание слов' 'Blurpy'
 'North Africa' 'Eastern European English, bulgarian english'
 'Transfemale - US English, American English, Transgender Woman - US English'
 'midwestern united states' 'European accent' 'Welsh English'
 'Singaporean English' 'Filipino' 'Malaysian English, Indonesian English'
 'Malaysian English']


In [ ]:


# Step 1: Define the function to generalize accents containing 'African' or 'South African'
def generalize_to_african_english(accent):
    # Check if 'African' or 'South African' is anywhere in the accent string
    if 'Northern English' in accent or 'Irish' in accent or 'England' in accent or 'Welsh' in accent:
        return 'British English'  # Generalize to 'African English'
    return accent  # Keep other accents unchanged

# Step 2: Apply the function to the 'accents' column in your DataFrame
metadata['accents'] = metadata['accents'].apply(generalize_to_african_english)

# Step 3: Print the unique accents to verify the changes
print("Generalized Accents in Metadata:")
print(metadata['accents'].unique())  # Display unique values after generalization


Generalized Accents in Metadata:
['American English' 'British English' 'Australian English'
 'South Asian English' 'European English' 'Canadian English' 'Icelandic'
 'Zambia' 'Slavic English'
 'Latin American accent influenced by American English'
 'New Zealand English' 'Algeria, Algerian' 'Italian' 'African English'
 'Slovenian, American English' 'East Asian English' 'South Asia English'
 'american' 'Turkish' 'Немного русский акцент произношения, съедание слов'
 'Blurpy' 'North Africa' 'Eastern European English, bulgarian english'
 'Transfemale - US English, American English, Transgender Woman - US English'
 'midwestern united states' 'European accent' 'Singaporean English'
 'Filipino' 'Malaysian English, Indonesian English' 'Malaysian English']


In [ ]:


# Step 1: Define the function to generalize accents containing 'African' or 'South African'
def generalize_to_african_english(accent):
    # Check if 'African' or 'South African' is anywhere in the accent string
    if 'Slavic' in accent or 'Italian' in accent or 'Eastern European English' in accent or 'Welsh' in accent:
        return 'European English'  # Generalize to 'African English'
    return accent  # Keep other accents unchanged

# Step 2: Apply the function to the 'accents' column in your DataFrame
metadata['accents'] = metadata['accents'].apply(generalize_to_african_english)

# Step 3: Print the unique accents to verify the changes
print("Generalized Accents in Metadata:")
print(metadata['accents'].unique())  # Display unique values after generalization


Generalized Accents in Metadata:
['American English' 'British English' 'Australian English'
 'South Asian English' 'European English' 'Canadian English' 'Icelandic'
 'Zambia' 'Latin American accent influenced by American English'
 'New Zealand English' 'Algeria, Algerian' 'African English'
 'Slovenian, American English' 'East Asian English' 'South Asia English'
 'american' 'Turkish' 'Немного русский акцент произношения, съедание слов'
 'Blurpy' 'North Africa'
 'Transfemale - US English, American English, Transgender Woman - US English'
 'midwestern united states' 'European accent' 'Singaporean English'
 'Filipino' 'Malaysian English, Indonesian English' 'Malaysian English']


In [ ]:


# Step 1: Define the function to generalize accents containing 'African' or 'South African'
def generalize_to_african_english(accent):
    # Check if 'African' or 'South African' is anywhere in the accent string
    if 'Singaporean' in accent or 'Filipino' in accent or 'Malaysian' in accent or 'Indonesian' in accent:
        return 'East Asian English'  # Generalize to 'African English'
    return accent  # Keep other accents unchanged

# Step 2: Apply the function to the 'accents' column in your DataFrame
metadata['accents'] = metadata['accents'].apply(generalize_to_african_english)

# Step 3: Print the unique accents to verify the changes
print("Generalized Accents in Metadata:")
print(metadata['accents'].unique())  # Display unique values after generalization


Generalized Accents in Metadata:
['American English' 'British English' 'Australian English'
 'South Asian English' 'European English' 'Canadian English' 'Icelandic'
 'Zambia' 'Latin American accent influenced by American English'
 'New Zealand English' 'Algeria, Algerian' 'African English'
 'Slovenian, American English' 'East Asian English' 'South Asia English'
 'american' 'Turkish' 'Немного русский акцент произношения, съедание слов'
 'Blurpy' 'North Africa'
 'Transfemale - US English, American English, Transgender Woman - US English'
 'midwestern united states' 'European accent']


In [ ]:


# Step 1: Define the function to generalize accents containing 'African' or 'South African'
def generalize_to_african_english(accent):
    # Check if 'African' or 'South African' is anywhere in the accent string
    if 'Latin American accent influenced by American English' in accent or 'Slovenian' in accent or 'US English' in accent or 'midwestern united states' in accent:
        return 'American English'  # Generalize to 'African English'
    return accent  # Keep other accents unchanged

# Step 2: Apply the function to the 'accents' column in your DataFrame
metadata['accents'] = metadata['accents'].apply(generalize_to_african_english)

# Step 3: Print the unique accents to verify the changes
print("Generalized Accents in Metadata:")
print(metadata['accents'].unique())  # Display unique values after generalization


Generalized Accents in Metadata:
['American English' 'British English' 'Australian English'
 'South Asian English' 'European English' 'Canadian English' 'Icelandic'
 'Zambia' 'New Zealand English' 'Algeria, Algerian' 'African English'
 'East Asian English' 'South Asia English' 'american' 'Turkish'
 'Немного русский акцент произношения, съедание слов' 'Blurpy'
 'North Africa' 'European accent']


In [ ]:


# Step 1: Define the function to generalize accents containing 'African' or 'South African'
def generalize_to_african_english(accent):
    # Check if 'African' or 'South African' is anywhere in the accent string
    if 'Zambia' in accent or 'Slovenian' in accent or 'Algeria' in accent or 'North Africa' in accent:
        return 'African English'  # Generalize to 'African English'
    return accent  # Keep other accents unchanged

# Step 2: Apply the function to the 'accents' column in your DataFrame
metadata['accents'] = metadata['accents'].apply(generalize_to_african_english)

# Step 3: Print the unique accents to verify the changes
print("Generalized Accents in Metadata:")
print(metadata['accents'].unique())  # Display unique values after generalization


Generalized Accents in Metadata:
['American English' 'British English' 'Australian English'
 'South Asian English' 'European English' 'Canadian English' 'Icelandic'
 'African English' 'New Zealand English' 'East Asian English'
 'South Asia English' 'american' 'Turkish'
 'Немного русский акцент произношения, съедание слов' 'Blurpy'
 'European accent']


In [ ]:


# Step 1: Define the function to generalize accents containing 'African' or 'South African'
def generalize_to_african_english(accent):
    # Check if 'African' or 'South African' is anywhere in the accent string
    if 'American English' in accent or 'american' in accent:
        return 'American English Accent'  # Generalize to 'African English'
    return accent  # Keep other accents unchanged

# Step 2: Apply the function to the 'accents' column in your DataFrame
metadata['accents'] = metadata['accents'].apply(generalize_to_african_english)

# Step 3: Print the unique accents to verify the changes
print("Generalized Accents in Metadata:")
print(metadata['accents'].unique())  # Display unique values after generalization


Generalized Accents in Metadata:
['American English Accent' 'British English' 'Australian English'
 'South Asian English' 'European English' 'Canadian English' 'Icelandic'
 'African English' 'New Zealand English' 'East Asian English'
 'South Asia English' 'Turkish'
 'Немного русский акцент произношения, съедание слов' 'Blurpy'
 'European accent']


In [ ]:


# Step 1: Define the function to generalize accents containing 'African' or 'South African'
def generalize_to_african_english(accent):
    # Check if 'African' or 'South African' is anywhere in the accent string
    if 'South Asian English' in accent or 'East Asian English' in accent:
        return 'Asian English Accent'  # Generalize to 'African English'
    return accent  # Keep other accents unchanged

# Step 2: Apply the function to the 'accents' column in your DataFrame
metadata['accents'] = metadata['accents'].apply(generalize_to_african_english)

# Step 3: Print the unique accents to verify the changes
print("Generalized Accents in Metadata:")
print(metadata['accents'].unique())  # Display unique values after generalization


Generalized Accents in Metadata:
['American English Accent' 'British English' 'Australian English'
 'Asian English Accent' 'European English' 'Canadian English' 'Icelandic'
 'African English' 'New Zealand English' 'South Asia English' 'Turkish'
 'Немного русский акцент произношения, съедание слов' 'Blurpy'
 'European accent']


In [ ]:


# Step 1: Define the function to generalize accents containing 'African' or 'South African'
def generalize_to_african_english(accent):
    # Check if 'African' or 'South African' is anywhere in the accent string
    if 'European accent' in accent or 'European English' in accent:
        return 'European English Accent'  # Generalize to 'African English'
    return accent  # Keep other accents unchanged

# Step 2: Apply the function to the 'accents' column in your DataFrame
metadata['accents'] = metadata['accents'].apply(generalize_to_african_english)

# Step 3: Print the unique accents to verify the changes
print("Generalized Accents in Metadata:")
print(metadata['accents'].unique())  # Display unique values after generalization


Generalized Accents in Metadata:
['American English Accent' 'British English' 'Australian English'
 'Asian English Accent' 'European English Accent' 'Canadian English'
 'Icelandic' 'African English' 'New Zealand English' 'South Asia English'
 'Turkish' 'Немного русский акцент произношения, съедание слов' 'Blurpy']


In [ ]:
# Check the frequency of each accent in the DataFrame
accent_counts = metadata['accents'].value_counts()

# Display the counts for the unclear accents
print("Frequency of unclear accents:")
for accent in ['American English Accent','Icelandic', 'Turkish', 'Немного русский акцент произношения, съедание слов', 'Blurpy']:
    print(f"{accent}: {accent_counts.get(accent, 0)}")


Frequency of unclear accents:
American English Accent: 719
Icelandic: 1
Turkish: 1
Немного русский акцент произношения, съедание слов: 1
Blurpy: 1


In [ ]:
# List of unclear accents to remove
unclear_accents = ['Icelandic', 'Turkish', 'Немного русский акцент произношения, съедание слов', 'Blurpy']

# Filter the DataFrame to exclude these accents
metadata = metadata[~metadata['accents'].isin(unclear_accents)]

# Display the unique accents to verify the changes
print("Accents after removing unclear categories:")
print(metadata['accents'].unique())


Accents after removing unclear categories:
['American English Accent' 'British English' 'Australian English'
 'Asian English Accent' 'European English Accent' 'Canadian English'
 'African English' 'New Zealand English' 'South Asia English']


In [ ]:


# Step 1: Define the function to generalize accents containing 'African' or 'South African'
def generalize_to_african_english(accent):
    # Check if 'African' or 'South African' is anywhere in the accent string
    if 'South Asia English' in accent:
        return 'Asian English Accent'  # Generalize to 'African English'
    return accent  # Keep other accents unchanged

# Step 2: Apply the function to the 'accents' column in your DataFrame
metadata['accents'] = metadata['accents'].apply(generalize_to_african_english)

# Step 3: Print the unique accents to verify the changes
print("Generalized Accents in Metadata:")
print(metadata['accents'].unique())  # Display unique values after generalization


Generalized Accents in Metadata:
['American English Accent' 'British English' 'Australian English'
 'Asian English Accent' 'European English Accent' 'Canadian English'
 'African English' 'New Zealand English']


In [ ]:
# Check the frequency of each accent in the DataFrame
accent_counts = metadata['accents'].value_counts()

# Display the counts for the unclear accents
print("Frequency of unclear accents:")
for accent in ['American English Accent','British English', 'Australian English', 'Asian English Accent', 'European English Accent','Canadian English','African English','New Zealand English']:
    print(f"{accent}: {accent_counts.get(accent, 0)}")


Frequency of unclear accents:
American English Accent: 719
British English: 394
Australian English: 139
Asian English Accent: 143
European English Accent: 105
Canadian English: 198
African English: 14
New Zealand English: 17


In [ ]:

# Step 1: Define a function to merge accents into broader categories
def merge_into_broad_categories(accent):
    # Merging into broader categories
    if accent in ['American English Accent', 'Canadian English']:
        return 'North American English'
    elif accent in ['British English', 'Australian English', 'New Zealand English']:
        return 'British English'
    elif accent == 'Asian English Accent':
        return 'Asian English'
    elif accent == 'European English Accent':
        return 'European English'
    elif accent == 'African English':
        return 'African English'
    else:
        return accent  # Keep the original if it doesn't match any of the above

# Step 2: Apply the function to merge categories in the 'accents' column of the DataFrame
metadata['broader_accents'] = metadata['accents'].apply(merge_into_broad_categories)

# Step 3: Print the resulting unique accents to verify the changes
print("Broader Categories for Accent Normalization:")
print(metadata['broader_accents'].value_counts())  # Display the count of each broader accent category


Broader Categories for Accent Normalization:
broader_accents
North American English    917
British English           550
Asian English             143
European English          105
African English            14
Name: count, dtype: int64


In [ ]:
metadata['broader_accents'].head()

,broader_accents
0,North American English
2,British English
4,British English
5,North American English
6,British English


In [ ]:
# Check if the 'accent' column is present in the dataset
if 'broader_accents' in metadata.columns:
    # Extract unique accents
    unique_accents = metadata['broader_accents'].dropna().unique()  # Drop NaN values and find unique accents
    print("Unique accents in the dataset:")
    print(unique_accents)  # Display the unique accents
else:
    print("The 'accent' column is not found in the metadata.")

Unique accents in the dataset:
['North American English' 'British English' 'Asian English'
 'European English' 'African English']


In [ ]:
# Step 3: Print the resulting unique accents to verify the changes
print("Broader Categories for Accent Normalization:")
print(metadata['broader_accents'].value_counts())  # Display the count of each broader accent category

Broader Categories for Accent Normalization:
broader_accents
North American English    917
British English           550
Asian English             143
European English          105
African English            14
Name: count, dtype: int64


In [ ]:
# Step 2: Specify the broader accent categories you want to work with
target_accents = ['North American English', 'British English', 'Asian English', 'European English', 'African English']

# Step 3: Filter the DataFrame based on the target accents
filtered_df = metadata[metadata['broader_accents'].isin(target_accents)]  # 'broader_accents' is the column with the accent labels

In [ ]:

# Display the first few rows to understand the structure
print(filtered_df.head())


                                           client_id  \
0  01e8ea298cdecf26e273f5baac3915eb992c493f229686...   
2  03b62f72067ec967c423852bef03d1b61e63c156d86f6e...   
4  05d33ad00cc2754da8e542a33a5255f9346535ef1d8619...   
5  08072f2de4dcc2bfec5058dca41eb9535b61ccd193ecc4...   
6  083af8bc921baf15ad5d8c8c876f4ecaf4f52bf6370161...   

                           path  \
0  common_voice_en_39751075.mp3   
2  common_voice_en_40087973.mp3   
4  common_voice_en_40117514.mp3   
5  common_voice_en_39603786.mp3   
6  common_voice_en_39603175.mp3   

                                         sentence_id  \
0  e5e7d4694b7160add018a08876327f254690c1ab4c39ea...   
2  e90c361c9684d01d31bc6e8df3060bc97e536ca707bef4...   
4  e9475052b6e625f8c5890389e4ffc17a1078dec1483592...   
5  e4657d8d47be955eb14e04cd1c2a2b9ef89d310f639678...   
6  e443f322884c5440d7f5072f21c5b0e1f0433ba6147471...   

                                            sentence sentence_domain  \
0  Madin was a significant figure of post-w

In [ ]:
# Extract relevant columns from the metadata
audio_files = filtered_df['path'].tolist()  # Get the audio file names
accents = filtered_df['broader_accents'].tolist()    # Get the corresponding accents

# Display the unique accents in the dataset
print("Unique accents in the dataset:", set(accents))


Unique accents in the dataset: {'Asian English', 'European English', 'African English', 'North American English', 'British English'}


In [ ]:
!pip install librosa

In [ ]:
import librosa
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

# Corrected path to processed audio files directory
audio_dir = os.path.join(extract_dir, 'en/input_audio_files')

# Function to extract MFCC features from an audio file
def extract_mfcc(file_path, n_mfcc=13):
    y, sr = librosa.load(file_path, sr=16000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return np.mean(mfcc.T, axis=0)

# Function for augmenting audio data
def augment_audio(file_path):
    y, sr = librosa.load(file_path, sr=16000)

    # Noise Injection
    noise = np.random.randn(len(y))
    y_noise = y + 0.005 * noise  # Adjust the noise factor as needed

    # Time Stretching
    y_stretch = librosa.effects.time_stretch(y, rate=1.1)  # Stretch by a factor of 1.1

    # Pitch Shifting
    y_pitch = librosa.effects.pitch_shift(y, sr=sr, n_steps=2)  # Corrected: Shift pitch up by 2 steps

    return [y_noise, y_stretch, y_pitch]

# Extract features and apply data augmentation
features = []
labels = []

for audio_file, accent in zip(audio_files, accents):
    try:
        file_path = os.path.join(audio_dir, audio_file)  # Corrected single path
        # Extract original MFCC features
        mfcc_features = extract_mfcc(file_path)
        features.append(mfcc_features)
        labels.append(accent)

        # Apply augmentations and extract MFCC features for each augmented audio
        for aug_audio in augment_audio(file_path):
            aug_mfcc_features = librosa.feature.mfcc(y=aug_audio, sr=16000, n_mfcc=13)
            aug_mfcc_mean = np.mean(aug_mfcc_features.T, axis=0)
            features.append(aug_mfcc_mean)
            labels.append(accent)
    except Exception as e:
        print(f"Error processing {audio_file}: {e}")

print("Feature extraction and augmentation completed!")

# Encode the accent labels to numeric values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

print("Encoded labels:", encoded_labels[:10])
print("Classes:", encoder.classes_)

# Split data into training and validation sets (80% training, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(features, encoded_labels, test_size=0.2, random_state=42)

class AccentDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Create PyTorch Datasets and DataLoaders for training and validation
train_dataset = AccentDataset(X_train, y_train)
val_dataset = AccentDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print("Training and Validation DataLoaders are ready!")

class AccentNormalizationModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(AccentNormalizationModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

# Initialize the model, loss function, and optimizer
model = AccentNormalizationModel(input_size=13, hidden_size=128, num_classes=len(encoder.classes_))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print("Model is initialized!")

# Training loop with validation
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for features, labels in train_loader:
        # Forward pass
        outputs = model(features.unsqueeze(1))  # Add extra dimension for LSTM
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Evaluate on the validation set
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for features, labels in val_loader:
            outputs = model(features.unsqueeze(1))
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Validation Accuracy: {val_accuracy:.2f}%')

print("Training with validation completed!")

def evaluate_model(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for features, labels in data_loader:
            outputs = model(features.unsqueeze(1))
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')

# Run evaluation on validation set
evaluate_model(model, val_loader)


Feature extraction and augmentation completed!
Encoded labels: [4 4 4 4 2 2 2 2 2 2]
Classes: ['African English' 'Asian English' 'British English' 'European English'
 'North American English']
Training and Validation DataLoaders are ready!
Model is initialized!
Epoch [1/10], Loss: 1.0950, Validation Accuracy: 62.79%
Epoch [2/10], Loss: 0.6849, Validation Accuracy: 67.92%
Epoch [3/10], Loss: 0.8347, Validation Accuracy: 68.93%
Epoch [4/10], Loss: 0.8784, Validation Accuracy: 71.24%
Epoch [5/10], Loss: 0.7010, Validation Accuracy: 71.39%
Epoch [6/10], Loss: 0.9247, Validation Accuracy: 67.56%
Epoch [7/10], Loss: 0.4752, Validation Accuracy: 73.19%
Epoch [8/10], Loss: 0.5128, Validation Accuracy: 71.03%
Epoch [9/10], Loss: 0.6110, Validation Accuracy: 72.69%
Epoch [10/10], Loss: 0.5938, Validation Accuracy: 73.55%
Training with validation completed!
Accuracy: 73.55%


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Function to evaluate the model using different metrics
def evaluate_model(model, data_loader):
    model.eval()
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for features, labels in data_loader:
            outputs = model(features.unsqueeze(1))
            _, predicted = torch.max(outputs, 1)

            all_labels.extend(labels.tolist())
            all_predictions.extend(predicted.tolist())

    # Calculate Accuracy
    accuracy = accuracy_score(all_labels, all_predictions)
    print(f'Accuracy: {accuracy * 100:.2f}%')

    # Generate Classification Report
    print("\nClassification Report:")
    print(classification_report(all_labels, all_predictions, target_names=encoder.classes_))

    # Generate Confusion Matrix
    print("\nConfusion Matrix:")
    print(confusion_matrix(all_labels, all_predictions))

# Run evaluation on validation set
evaluate_model(model, val_loader)


Accuracy: 73.55%

Classification Report:
                        precision    recall  f1-score   support

       African English       0.00      0.00      0.00        11
         Asian English       0.81      0.35      0.49       119
       British English       0.70      0.70      0.70       451
      European English       0.74      0.63      0.68        76
North American English       0.75      0.84      0.79       727

              accuracy                           0.74      1384
             macro avg       0.60      0.51      0.53      1384
          weighted avg       0.73      0.74      0.72      1384


Confusion Matrix:
[[  0   0   4   1   6]
 [  0  42  26   5  46]
 [  0   5 316   4 126]
 [  0   1   4  48  23]
 [  0   4 104   7 612]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
